# Title: Off_Period_Detection_Rolling
## Description: 특정 윈도우(Window) 기간 동안 연속으로 값이 0인 구간을 탐지하여 비활성 상태로 마킹함.
## Input: 
- pandas DataFrame
- group col
- date col
- traget col
- window length
## Output: 
- 'is_off_period' (비활성 여부) 컬럼이 추가된 DataFrame
## Check Point: 
- Memory Usage: 대용량 데이터의 경우 df.copy()가 메모리를 추가로 점유할 수 있습니다.
- Sorting: 반환되는 데이터는 [group_col, date_col] 기준으로 정렬된 상태입니다.

In [ ]:
import pandas as pd
import numpy as np

def add_rolling_zero_period_vectorized(df, group_col, date_col, target_col, window=7):
    """
    Groupby와 Transform을 활용한 고속 휴점 탐지 로직
    """
    # 1. 원본 데이터 침해 방지를 위한 명시적 깊은 복사
    # 파이프라인 중간에 원본이 변형되는 것을 막습니다.
    df = df.copy()

    # 2. 정렬 수행 (시계열 롤링 계산을 위해 필수)
    # reset_index(drop=True)를 제거하여 원본 인덱스 정보를 보존합니다.
    df = df.sort_values([group_col, date_col])

    # 3. 그룹별 연속 구간 탐지 (Vectorized Logic)
    # 데이터가 정렬되어 있으므로 shift()를 통해 이전 행과 비교 가능
    is_zero = df[target_col] == 0
    
    # 상태가 변하거나(0<->1), 그룹이 변하는 지점 탐지
    condition_change = (is_zero != is_zero.shift(1))
    group_change = (df[group_col] != df[group_col].shift(1))
    
    # run_id: 연속된 상태가 유지되는 구간마다 고유 ID 부여
    run_id = (condition_change | group_change).cumsum()
    
    # 4. 각 구간별 길이 계산 (Transform 활용)
    # groupby().transform('count')는 집계 결과를 원본 행의 개수만큼 브로드캐스팅합니다.
    run_lengths = df.groupby(run_id)[target_col].transform('count')
    
    # 5. 최종 마킹
    # 값이 0이면서 & 연속된 길이가 window 이상인 경우
    df['휴점여부'] = np.where(is_zero & (run_lengths >= window), 1, 0)

    return df

## How to Use:
- `window` 파라미터는 비즈니스 도메인에 맞춰 설정하세요. (예: 주간 패턴이 강하면 7일)
- 매출 데이터뿐만 아니라, 센서 데이터의 '장비 가동 중지' 구간 탐지 등에도 응용 가능합니다.
- `add_rolling_zero_period_vectorized(df, 'store_id', 'date', 'sales', window=7)` 형태로 호출합니다.
- `reset_index(drop=True)`가 포함되어 있으므로, 원본 인덱스가 중요하다면 함수 호출 전 인덱스를 컬럼으로 빼두세요.

## Troubleshooting:
- **메모리 부족**: `groupby` 연산조차 무거운 수억 건의 데이터라면, `group_col`을 기준으로 데이터를 쪼개서(chunk) 처리해야 합니다.
- **결과 불일치**: 반드시 `date_col` 기준으로 정렬되어 있어야 합니다. 정렬되지 않은 시계열에서 `shift()`는 무의미합니다.
- Index 중복: 만약 원본 데이터의 Index가 고유(Unique)하지 않다면 groupby 연산 등에서 의도치 않은 경고가 발생할 수 있습니다. 가능한 reset_index()로 인덱스를 정리한 후 입력하거나, 고유한 인덱스를 보장하는 것이 좋습니다.
- 자료형 이슈: shift() 연산 시 target_col에 NaN이 포함되어 있다면 결과가 부정확할 수 있습니다. 사전에 fillna(0) 처리를 권장합니다.